In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
pd.options.display.max_rows = 99
import seaborn as sns

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' #'last_expr'

base_path = '/kaggle/input/tabular-playground-series-apr-2022/'
df_train_lbl = pd.read_csv(f'{base_path}/train_labels.csv')
df_train = pd.read_csv(f'{base_path}/train.csv')
df_train = df_train.merge(df_train_lbl, on=['sequence'], how='left')
sensor_cols = df_train.columns[3:-1].to_list()

# Intro

This Notebook shows 63 sequences that have at aleast 1 sensor (excluding sensor_02 as I think this may be 'normal' for it) 'stuck' at the same value for all steps in a sequence. Including sensor_02 we will get 69 sequences.
</br> 44 out of those have 8 or more sensor stuck through out the entire sequence. Based on this and how the data for other sensors look like for me they are clear outliers that should be removed.
</br> I know the data is generated, but I think this can simulate a record where system was recoring data when sensors were not connected.

In [ ]:
df_train_unique_1 = df_train.drop(['subject', 'step', 'state', 'sensor_02'], axis=1).groupby(['sequence']).agg(lambda x: x.nunique() == 1).sum(axis=1).sort_values(ascending=False)

In [ ]:
at_least_1 = df_train_unique_1[df_train_unique_1>0]
'Sequences with at least 1 sensor stuck: ', len(at_least_1)
at_least_1

In [ ]:
at_least_8 = df_train_unique_1[df_train_unique_1>1]
'Sequences with at least 8 sensor stuck: ', len(at_least_8)
at_least_8

In [ ]:
df_plt = df_train[df_train.sequence.isin( df_train_unique_1[df_train_unique_1>0].index )].melt(id_vars=['sequence', 'subject', 'step'], value_vars=sensor_cols, var_name='sensor')

relplot_by_seq = lambda data, seq: sns.relplot(kind="line", data=data[data.sequence == seq], x='step', y = 'value', col='sensor', facet_kws={'sharey': False, 'sharex': False}, height=1.8, aspect=2)
for seq in df_plt.sequence.unique():
    _ = relplot_by_seq(df_plt, seq)

<b>The axis are very small, but they are here just to show you wherre the flat-lines are.<b>